In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen1.5-0.5B-Chat")
%load_ext autoreload
%autoreload 2

import os
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {} '.format(device))

/home/louzix/Documents/project/Latent-truth-detector/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Device: cpu 


In [2]:
from transformers import set_seed
set_seed(42)

In [48]:
prompt = "the city of Marseille is in France" 

prompt2 = "the city of Marseille is in Russia"
message = [
    {"role": "system", "content": "You are a helpful assistant and will only answer with False or True."},
    {"role": "user", "content": prompt2}
]

In [51]:
text = tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=20,
    do_sample=False,
    top_p=1.0,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

False. Marseille, also known as Marseille Bay, is located in France, not Russia. It is


In [ ]:
#false
tensor(0.5568)
tensor(-0.9571)
tensor(-0.1110)
tensor(-0.0090)
tensor(-14.5962)
#true-14.5962
tensor(0.3529)
tensor(-0.7512)
tensor(-0.0089)
tensor(0.0538)
tensor(-14.5962)

In [5]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (o_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (up_proj): Linear(in_features=1024, out_features=2816, bias=False)
          (down_proj): Linear(in_features=2816, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Line

In [44]:
#308 12
#773 12
#875 15
# 4  12
#543 12

#create a hook that will modify the value of a neuron in the layer 14
def hook_fn1(module, input, output):
    print(output[0][0][-1][308])
    #output[0][0][-1][308]=0
    print(output[0][0][-1][773])
    #output[0][0][-1][773]=0
    print(output[0][0][-1][4])
    #output[0][0][-1][4]=-0
    print(output[0][0][-1][543])
    #output[0][0][-1][543]=1
    return output

def hook_fn2(module, input, output):
    print(output[0][0][0][111])
    print(output[0][0][0][1020])
    #output[0][0][0][875]=-14.5962
    return output
    
def hook_fn3(module, input, output):
    print(output[0][0][0][7])
    #output[0][0][0][7]=0
    print(output[0][0][0][7])
    return output

def hook_fn4(module, input, output):
    print(output[0][0][0][7])
    #output[0][0][0][7]=0
    #output[0][0][0][774]=-0.2
    print(output[0][0][0][774])
    return output

def hook_fn5(module, input, output):
    print(output[0][0][0][7])
    #output[0][0][0][7]=0
    #output[0][0][0][774]=-0.2
    print(output[0][0][0][898])
    return output
    
#register the hook
hook1 = model.model.layers[12].register_forward_hook(hook_fn1)
hook2 = model.model.layers[15].register_forward_hook(hook_fn2)
hook3 = model.model.layers[7].register_forward_hook(hook_fn3)
hook4 = model.model.layers[10].register_forward_hook(hook_fn4)
hook5 = model.model.layers[22].register_forward_hook(hook_fn5)

In [50]:
hook1.remove()
hook2.remove()
hook3.remove()
hook4.remove()
hook5.remove()